Common setup and functions used by the CW305 ECC demos.

In [ ]:
# Basic initialization:
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "disabled"

if PLATFORM == 'CWPRO':
    scope.adc.stream_mode = True
    scope.adc.samples = 1200000
    target.pll.pll_outfreq_set(10E6, 1)
    target._clksleeptime = 150
    scope.gain.db = 20
elif PLATFORM == 'CWHUSKY':
    scope.adc.stream_mode = True
    scope.adc.samples = 1200000
    target.pll.pll_outfreq_set(15E6, 1)
    target._clksleeptime = 100
    scope.gain.db = 20
elif PLATFORM == 'CWLITE':
    scope.adc.samples = 24400
    target.pll.pll_outfreq_set(50E6, 1)
    target._clksleeptime = 30
    scope.gain.db = 30

if PLATFORM == 'CWHUSKY':
    scope.adc.offset = 3
else:
    scope.adc.offset = 0

target.vccint_set(1.0)
# we only need PLL1:
target.pll.pll_enable_set(True)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(True, 1)
target.pll.pll_outenable_set(False, 2)

if PLATFORM == 'CWHUSKY':
    scope.clock.clkgen_freq = 15e6
    scope.clock.clkgen_src = 'extclk'
    scope.clock.adc_mul = 1
else:
    scope.clock.adc_src = "extclk_x1"

In [ ]:
import numpy as np
cycles = np.load('data/ecc_cycles.npy')

In [ ]:
def set_adc(samples):
    if PLATFORM == 'CWPRO':
        scope.adc.stream_mode = True
        scope.adc.samples = samples
        scope.adc.offset = 0
    elif PLATFORM == 'CWHUSKY':
        scope.adc.stream_mode = True
        scope.adc.samples = samples
        scope.adc.offset = 3
        scope.adc.segments = 1
    elif PLATFORM == 'CWLITE':
        scope.adc.samples = 24400
        scope.adc.offset = 0

In [ ]:
def random_k(bits=256, tries=100):
    import random
    for i in range(tries):
        k = random.getrandbits(bits)
        if k < target.curve.order and k > 0:
            return k
    raise ValueError("Failed to generate a valid random k after %d tries!" % self.tries)

In [ ]:
from chipwhisperer.common.traces import Trace
from tqdm import tnrange
import numpy as np
import time
import math 

def get_traces(N=50, k=0, randomize_k=False, samples=1200000):
    traces = []
    attempt4 = get_bitfile_version() == 'attempt4'
    set_adc(samples)
    if PLATFORM == 'CWPRO' or PLATFORM == 'CWHUSKY':
        for i in tnrange(N, desc='Capturing traces'):
            P = target.new_point() # every trace uses a different point
            if randomize_k:
                k = random_k()
            assert k != 0
            if attempt4:
                kb = 0x10000000000000000000000000000000000000000000000000000000000000000 - k
                target.fpga_write(0x12, list(int.to_bytes(kb, length=32, byteorder='little')))

            ret = target.capture_trace(scope, Px=P.x, Py=P.y, k=k, check=True)
            if not ret:
                print("Failed capture")
                continue
            traces.append(ret)

    elif PLATFORM == 'CWLITE':
        segments = math.ceil(target.pmul_cycles / scope.adc.samples)
        #segments = 1
        for i in tnrange(N, desc='Capturing traces'):
            scope.adc.offset = 0
            wave = np.array([])
            P = target.new_point() # every trace uses a different point
            if randomize_k:
                k = random_k()
            assert k != 0
            if attempt4:
                kb = 0x10000000000000000000000000000000000000000000000000000000000000000 - k
                target.fpga_write(0x12, list(int.to_bytes(kb, length=32, byteorder='little')))

            for j in range(segments):
                ret = target.capture_trace(scope, Px=P.x, Py=P.y, k=k)
                if not ret:
                    print("Failed capture")
                    continue
                wave = np.append(wave, ret.wave)
                scope.adc.offset += scope.adc.samples

            traces.append(Trace(wave[1:], ret.textin, ret.textout, None))

    return traces

In [ ]:
def get_sums(traces, poi):
    sums = []
    for c in cycles:
        sum = 0
        for trace in traces:
            for i in poi:
                power = trace.wave[c+abs(i)]
                if i < 0:
                    sum -= power
                else:
                    sum += power
        sums.append(sum/len(traces))
    return sums

In [ ]:
def get_corrected_sums(traces, poi):
    sums = []
    for c in range(len(cycles)-1):
        sum = 0
        for trace in traces:
            for p in poi:
                # shortcut: use the ~halfway point to determine whether the leakage influences the current bit or not
                if abs(p) > 2000:
                    power = trace.wave[cycles[c]+abs(p)]
                else:
                    power = trace.wave[cycles[c+1]+abs(p)]
                if p < 0:
                    sum -= power
                else:
                    sum += power

        sums.append(sum/len(traces))
    return sums

In [ ]:
def get_corrs(traces):

    corrsxonly = []
    corrsyonly = []
    corrszonly = []
    corrsall = []

    for i in range (0, len(cycles)-1):
        corrx = 0
        corry = 0
        corrz = 0

        for trace in traces:
            corrx += np.corrcoef(trace.wave[cycles[i]+rupdate_offset:cycles[i]+rupdate_offset+rupdate_cycles], trace.wave[cycles[i+1]+rxread_offset:cycles[i+1]+rxread_offset+rupdate_cycles])[0][1]
            corry += np.corrcoef(trace.wave[cycles[i]+rupdate_offset:cycles[i]+rupdate_offset+rupdate_cycles], trace.wave[cycles[i+1]+ryread_offset:cycles[i+1]+ryread_offset+rupdate_cycles])[0][1]
            #corrz += np.corrcoef(trace.wave[cycles[i]+rupdate_offset:cycles[i]+rupdate_offset+rupdate_cycles], trace.wave[cycles[i+1]+rzread_offset:cycles[i+1]+rzread_offset+rupdate_cycles])[0][1]

        #corrsall.append((corrx+corry+corrz)/len(traces))
        # consider only Y component for attack; uncomment above to study effect of other X/Z components:
        corrsall.append(corry/len(traces))
        #corrsall.append((corry+corrx)/len(traces))
    return corrsall

In [ ]:
def poi_guess(metric, thresholds):
    poi_init_threshold, poi_reg_threshold = thresholds
    guess = ''
    if get_bitfile_version() == 'attempt4':
        start = 1
        initial = False
    else:
        start = 0
        initial = True
    for kbit in range(start,255):
        if initial:
            if metric[kbit] < poi_init_threshold:
                guess += '0'
            else:
                guess += '1'
                initial = False
        else:
            if metric[kbit] < poi_reg_threshold:
                guess += '0'
            else:
                guess += '1'
            
    return guess

In [ ]:
def corr_guess(metric):
    initial = True
    guess = ''
    for kbit in range(0,255):
        if initial:
            if metric[kbit] > corr_init_threshold:
                guess += '0'
            else:
                guess += '1'
                initial = False
        else:
            if metric[kbit] > corr_reg_threshold:
                guess += '0'
            else:
                guess += '1'

    return guess

In [ ]:
def check_guess(guess, k, verbose=False):
    guesses = []
    if get_bitfile_version() == 'attempt4':
        top = 254
        for a in (['0', '1']):
            for b in (['0', '1']):
                guesses.append(int(a + guess + b, 2))
    else:
        top = 255
        guesses = [int(guess + '0', 2), int(guess + '1', 2)]

    if k in guesses:
        return ('Guessed right!', 0, 0)
    else:
        wrong_bits = []
        for kbit in range(top):
            if int(guess[kbit]) != ((k >> (top-kbit)) & 1):
                wrong_bits.append(top-kbit)
        if verbose:
            print('Attack failed.')
            print('Guesses: %s' % hex(guesses[0]))
            for guess in guesses[1:]:
                print('         %s' % hex(guess))
            print('Correct: %s' % hex(k))
            print('Wrong bits: %s' % wrong_bits)
        return ('Failed: %3d wrong bits' % len(wrong_bits), len(wrong_bits), wrong_bits)

In [ ]:
def poi_guess_threshold(metric, distance_threshold, thresholds):
    poi_init_threshold, poi_reg_threshold = thresholds
    guess = ''
    guessed_bits = []
    distances = []
    
    if get_bitfile_version() == 'attempt4':
        start = 1
        initial = False
    else:
        start = 0
        initial = True
    
    if distance_threshold <= 0:
        raise ValueEror("Threshold must be greater than 0")
        
    #1. Calculate distances from decision thresholds:
    for kbit in range(0,255):
        if initial:
            distances.append(abs(metric[kbit]- poi_init_threshold))
        else:
            distances.append(abs(metric[kbit]- poi_reg_threshold))

    #2. Calculate the mininum distance from decision threshold for which we'll enter a guess:
    avg = np.average(distances)
    top = max(distances)
    base = top-avg
    distance_threshold = distance_threshold * base

    #3. 
    if get_bitfile_version() == 'attempt4':
        initial = False
    else:
        initial = True
    for kbit in range(start,255):
        if initial:
            if abs(metric[kbit] - poi_init_threshold) > distance_threshold:
                guessed_bits.append(kbit)
            else:
                pass
            if metric[kbit] > poi_init_threshold:
                guess += '0'
            else:
                guess += '1'
                initial = False
        else:
            if abs(metric[kbit] - poi_reg_threshold) > distance_threshold:
                guessed_bits.append(kbit)
            else:
                pass
            if metric[kbit] < poi_reg_threshold:
                guess += '0'
            else:
                guess += '1'
    
    return guess, guessed_bits

In [ ]:
def get_trace_segments(N=50, poi=[-6, 7, 4202, -4203], randomize_k=False, k=0, husky_timed_segments=True):
    trace_segments = []
    attempt4 = get_bitfile_version() == 'attempt4'
    if PLATFORM == 'CWPRO' or (PLATFORM == 'CWHUSKY' and not husky_timed_segments): # note this approach can be used for Husky as well, but the segmented capture is faster!:
        scope.adc.segments = 1
        scope.adc.segment_cycles = 0
        scope.adc.stream_mode = True
        scope.adc.samples = 1200000
        scope.adc.offset = 3
        
        for i in tnrange(N, desc='Capturing traces'):
            P = target.new_point() # every trace uses a different point
            
            if randomize_k:
                k = random_k()
            assert k != 0
            if attempt4:
                kb = 0x10000000000000000000000000000000000000000000000000000000000000000 - k
                target.fpga_write(0x12, list(int.to_bytes(kb, length=32, byteorder='little')))

            ret = target.capture_trace(scope, Px=P.x, Py=P.y, k=k)
            if not ret:
                print("Failed capture")
                continue
            trace_segment = []
            for c in cycles:
                for p in poi:
                    trace_segment.append(ret.wave[c+abs(p)])
            trace_segments.append(Trace(trace_segment, ret.textin, ret.textout, None))
            
    elif PLATFORM == 'CWHUSKY':
        scope.adc.stream_mode = False
        scope.adc.segments = 256
        scope.adc.segment_cycles = 4204
        scope.adc.samples = 11
        scope.adc.offset = int(cycles[0] + 4201 + 3)
        if poi == [-6, 7, 4202, -4203]:
            indices = [1, 2, 9, 10]
        elif poi == [-6, 7, 4201, -4202]:
            indices = [0, 1, 9, 10]
        elif poi == [-6, 7]:
            indices = [9, 10]
        else:
            raise ValueError("Sorry, Husky timed segments only work for a specific set of markers; either set husky_timed_segments=False, or write your own segmented capture function" % poi)

        for i in tnrange(N, desc='Capturing traces'):
            P = target.new_point() # every trace uses a different point
            
            if randomize_k:
                k = random_k()
            assert k != 0
            if attempt4:
                kb = 0x10000000000000000000000000000000000000000000000000000000000000000 - k
                target.fpga_write(0x12, list(int.to_bytes(kb, length=32, byteorder='little')))
            
            ret = target.capture_trace(scope, Px=P.x, Py=P.y, k=k)
            if not ret:
                print("Failed capture")
                continue
            trace_segment = [0, 0] # first two samples are missed but that's inconsequential since they provide no useful side channel leakage
            for j,c in enumerate(cycles):
                base = scope.adc.samples*j
                for i,p in enumerate(poi):
                    trace_segment.append(ret.wave[base+indices[i]])
            trace_segments.append(Trace(trace_segment, ret.textin, ret.textout, None))

    elif PLATFORM == 'CWLITE':
        print('Not implemented for CW-lite')

    return trace_segments

In [ ]:
def get_segment_sums(trace_segments, poi):
    # Note: crucial that poi be identical to that used for get_trace_segments! (including order of elements)
    sums = []
    npois = len(poi)
    for c in range(len(cycles)-1):
        sum = 0
        for segment in trace_segments:
            for i,p in enumerate(poi):
                # shortcut: use the ~halfway point to determine whether the leakage influences the current bit or not
                if abs(p) > 2000:
                    base = c*npois
                else:
                    base = (c+1)*npois
                if p > 0:
                    sum += segment.wave[base+i]
                else:
                    sum -= segment.wave[base+i]
        sums.append(sum/len(trace_segments))
    return sums

In [ ]:
def change_bitfile(VERSION):
    if VERSION == 'original' or VERSION == 'attempt1' or VERSION == 'attempt2' or VERSION == 'attempt3' or VERSION == 'attempt4':
        bsfile = '../../../hardware/victims/cw305_artixtarget/fpga/vivado_examples/ecc_p256_pmul/bitfiles/' + VERSION + '.bit'
    else:
        raise ValueError("unsupported version %s" % VERSION)
    
    if VERSION == get_bitfile_version():
        pass
    else:
        status = target.fpga.FPGAProgram(open(bsfile, "rb"), exceptOnDoneFailure=False)
        assert status, "FPGA Done pin failed to go high"

        if VERSION == 'original':
            assert (target.get_fpga_buildtime() == 'FPGA build time: 10/13/2020, 9:31')
        elif VERSION == 'attempt1':
            assert (target.get_fpga_buildtime() == 'FPGA build time: 3/7/2021, 22:35')
        elif VERSION == 'attempt2':
            assert (target.get_fpga_buildtime() == 'FPGA build time: 3/17/2021, 14:54')
        elif VERSION == 'attempt3':
            assert (target.get_fpga_buildtime() == 'FPGA build time: 3/19/2021, 13:41')
        elif VERSION == 'attempt4':
            assert (target.get_fpga_buildtime() == 'FPGA build time: 3/29/2021, 22:46')

        if PLATFORM == 'CWHUSKY':
            # on Husky, reloading the FPGA will cause Husky's external clock frequency monitor to flag an error:
            import time
            time.sleep(0.5)
            scope.errors.clear()

In [ ]:
def get_bitfile_version():
    if target.get_fpga_buildtime() == 'FPGA build time: 10/13/2020, 9:31':
        return "original"
    elif target.get_fpga_buildtime() == 'FPGA build time: 3/7/2021, 22:35':
        return "attempt1"
    elif target.get_fpga_buildtime() == 'FPGA build time: 3/17/2021, 14:54':
        return "attempt2"
    elif target.get_fpga_buildtime() == 'FPGA build time: 3/19/2021, 13:41':
        return "attempt3"
    elif target.get_fpga_buildtime() == 'FPGA build time: 3/29/2021, 22:46':
        return "attempt4"
    else:
        raise ValueError("Unrecognized version")

In [ ]:
def consecutives(trace_segments, poi, distance_threshold, thresholds):

    wrong_bits = []
    solid_guessed_bits = []
    total_wrong_bits = 0
    total_solid_guessed_bits = 0
    total_right_solid_guesses = 0
    total_wrong_solid_guesses = 0
    correct_solid_guesses = []
    all_wrong_bits = []

    print('Computing averages...')
    for trace_segment in trace_segments:
        sums = get_segment_sums([trace_segment], poi)

        guess, tguessed_bits = poi_guess_threshold(sums, distance_threshold, thresholds)
        (status, num_wrong_bits, twrong_bits) = check_guess(guess, trace_segment.textin['k'])

        total_wrong_bits += num_wrong_bits
        all_wrong_bits.append(num_wrong_bits)
        total_solid_guessed_bits += len(tguessed_bits)

        wrong_solid_guesses = len(set(twrong_bits) & set(tguessed_bits))
        right_solid_guesses = len(tguessed_bits) - wrong_solid_guesses

        total_wrong_solid_guesses += wrong_solid_guesses
        total_right_solid_guesses += right_solid_guesses

        wrong_bits.append(twrong_bits)
        solid_guessed_bits.append(tguessed_bits)

        correct_solid_guesses.append(list(set(tguessed_bits) - set(twrong_bits)))

    print('All results are per-trace averages:')
    print('Average number of wrong bits (all guesses):     %5.1f' % (total_wrong_bits/len(trace_segments)))
    print('Average number of solid guessed bits:           %5.1f' % (total_solid_guessed_bits/len(trace_segments)))
    print('Average number of correct solid guessed bits:   %5.1f' % (total_right_solid_guesses/len(trace_segments)))
    print('Average number of incorrect solid guessed bits: %5.1f' % (total_wrong_solid_guesses/len(trace_segments)))

    print('Computing number of good traces...')
    # stats when taking only what we think are good guesses
    min_c_len = 3 # we only care about at least this many correct consecutive guesses
    total_good_consecutives = 0
    total_bad_consecutives = 0
    all_run_counts = np.zeros(255, np.int16)
    good_traces = 0
    bad_good_traces = 0
    good_trace_ids = []
    for t in range(len(trace_segments)):
        run_counts = np.zeros(255, np.int16)
        good_trace = False
        bad_good_trace = False

        # now we look for consecutive guesses, among the list of good *and* bad guesses - then we'll flag whether any bad guesses snuck in there
        guesses = np.sort(solid_guessed_bits[t])
        consecutives = np.split(guesses, np.where(np.diff(guesses) != 1)[0]+1)
        good_consecutives = 0
        bad_consecutives = 0
        for i,c in enumerate(consecutives):
            if len(c) >= min_c_len:
                if any(x in consecutives[i] for x in wrong_bits[t]):
                    bad_consecutives += 1
                    bad_good_trace = True
                else:
                    good_consecutives += 1
                    run_counts[len(c)] += 1
                    if len(c) >= 5:
                        good_trace = True
        total_good_consecutives += good_consecutives
        total_bad_consecutives += bad_consecutives
        all_run_counts += run_counts
        if run_counts[3] >= 3 or run_counts[4] >= 2:
            good_trace = True
        if good_trace:
            good_traces += 1
            good_trace_ids.append(t)
            if bad_good_trace:
                bad_good_traces += 1

    print("Total good consecutives: %3d (%5.2f per traces)" % (total_good_consecutives, float(total_good_consecutives/len(trace_segments))))
    print("Total bad consecutives: %3d (%5.2f per traces)" % (total_bad_consecutives, float(total_bad_consecutives/len(trace_segments))))
    print('Number of good traces: %d' % good_traces)
    print('Number of BAD good traces: %d' % bad_good_traces)